In [1]:
#pip install wget
import pandas as pd
import numpy as np
import json as js
import requests

In [8]:
# 1
ru = requests.get("https://fakestoreapi.com/users")
open("users.json", "wb").write(ru.content)
rc = requests.get("https://fakestoreapi.com/carts/?startdate=2000-01-01&enddate=2023-04-07")
open("carts.json", "wb").write(rc.content)
rp = requests.get("https://fakestoreapi.com/products")
open("products.json", "wb").write(rp.content)

print("Data has been retrieved. It is stored in files named: 'users.json', 'carts.json' and 'products.json'.")

Data has been retrieved. It is stored in files named: 'users.json', 'carts.json' and 'products.json'.


In [3]:
with open('users.json', 'r') as f:
    users = js.load(f)

ids = np.array([])
fnames = np.array([])
lnames = np.array([])
    
for u in users:
    ids = np.append(ids, u['id'])
    fnames = np.append(fnames, u['name']['firstname'])
    lnames = np.append(lnames, u['name']['lastname'])

dfu = pd.DataFrame({'id':ids, 'first name':fnames, 'last name':lnames})
dfu['id'] = dfu['id'].astype(int)
dfu.set_index('id', drop=True, inplace=True)
dfu

,first name,last name
id,,
1,john,doe
2,david,morrison
3,kevin,ryan
4,don,romer
5,derek,powell
6,david,russell
7,miriam,snyder
8,william,hopkins
9,kate,hale


In [4]:
# 2
with open('products.json', 'r') as f:
    products = js.load(f)

ids = np.array([])
categories = np.array([])
prices = np.array([])

for p in products:
    ids = np.append(ids, p['id'])
    categories = np.append(categories, p['category'])
    prices = np.append(prices, p['price'])

dfp = pd.DataFrame({'id':ids, 'category':categories, 'price':prices})
dfp['id'] = dfp['id'].astype(int)
dfp.set_index('id', drop=True, inplace=True)
print(dfp.groupby('category').sum()['price'])

category
electronics         1994.99
jewelery             883.98
men's clothing       204.23
women's clothing     157.72
Name: price, dtype: float64


In [5]:
# 3
with open('carts.json', 'r') as f:
    carts = js.load(f)

cid = 0
value = 0
uid = 0

for c in carts:
    t = 0
    for i in c['products']:
        t += i['quantity'] * dfp['price'][i['productId']]
    if t > value:
        value = t
        cid = c['id']
        uid = c['userId']

print(f"Cart with id {cid} has the highest value of: {value}. It belongs to the owner: {dfu['first name'][uid]} {dfu['last name'][uid]}.")

Cart with id 2 has the highest value of: 2578.7. It belongs to the owner: john doe.


In [6]:
# 4
uid1 = 0
uid2 = 0
dist = 0

for u1 in users:
    for u2 in users:
        lat = abs(float(u1['address']['geolocation']['lat']) - float(u2['address']['geolocation']['lat']))
        lon = abs(float(u1['address']['geolocation']['long']) - float(u2['address']['geolocation']['long']))
        d = np.sqrt(lat*lat + lon*lon)
        if(d>dist):
            uid1 = u1['id']
            uid2 = u2['id']
            dist = d

print(f"Users with ids: {uid1} and {uid2} live furthest away from each other. Their names are respectively: {dfu['first name'][uid1]} {dfu['last name'][uid1]} and {dfu['first name'][uid2]} {dfu['last name'][uid2]}")

Users with ids: 1 and 5 live furthest away from each other. Their names are respectively: john doe and derek powell
